# Imports

In [ ]:
from ase import Atoms

In [ ]:
from ase.build import bulk

In [ ]:
from ase.calculators.morse import MorsePotential

In [ ]:
from pyiron_contrib.tinybase.ase import AseStaticTask, AseMDTask, AseMinimizeTask

In [ ]:
from pyiron_contrib.tinybase.murn import MurnaghanTask

In [ ]:
from pyiron_contrib.tinybase.executor import ProcessExecutor, BackgroundExecutor, Executor

In [ ]:
import logging
logging.getLogger().setLevel(20)

# Simple ASE Static Calculation

In [ ]:
a = AseStaticTask()

In [ ]:
a.input.structure = bulk("Fe")

In [ ]:
a.input.calculator = MorsePotential()

In [ ]:
ret, output = a.execute(); ret, output

In [ ]:
output.energy_pot

In [ ]:
exe = a.run()
exe.output[0].energy_pot

# ASE MD

In [ ]:
md = AseMDTask()

In [ ]:
md.input.structure = bulk("Fe", a=1.2, cubic=True).repeat(3)
md.input.calculator = MorsePotential()

In [ ]:
md.input.steps = 100
md.input.timestep = 3
md.input.temperature = 600
md.input.output_steps = 20

In [ ]:
%%time
exe = md.run(how='process')

In [ ]:
exe.wait()

In [ ]:
exe._run_machine._data

In [ ]:
exe._run_machine.state

In [ ]:
exe._run_time

In [ ]:
exe._collect_time

In [ ]:
exe.output[0].plot_energies()

In [ ]:
exe.output[0].animate_structures()

# ASE Minimize

In [ ]:
from pyiron_atomistics import ase_to_pyiron

In [ ]:
mi = AseMinimizeTask()

In [ ]:
dimer = Atoms(symbols=['Fe', 'Fe'], positions=[[0,0,0], [0,0, .75]], cell=[10,10,10])

In [ ]:
mi.input.structure = dimer
mi.input.calculator = MorsePotential(rcut1=6,rcut2=10)

In [ ]:
ase_to_pyiron(mi.input.structure).plot3d()

In [ ]:
mi.input.max_steps = 100
mi.input.output_steps = 1
mi.input.ionic_force_tolerance = 1e-6

In [ ]:
mi.input.gpmin()

In [ ]:
%%time
exe = mi.run(how='foreground')

In [ ]:
exe.status

In [ ]:
abs(exe.output[0].forces[-1]).max()

In [ ]:
exe._run_machine.state

In [ ]:
exe._run_time

In [ ]:
exe._collect_time

In [ ]:
exe.output[0].plot_energies()

In [ ]:
exe.output[0].pot_energies

In [ ]:
exe.output[0].animate_structures()

# Murnaghan

## Basic

In [ ]:
m = MurnaghanTask()

In [ ]:
m.input.task = AseStaticTask()
m.input.task.input.calculator = MorsePotential()
m.input.structure = bulk("Fe", a=1.2)

In [ ]:
m.input.task.input

In [ ]:
m.input.set_strain_range(.5, 50)

In [ ]:
m.input.strains

In [ ]:
m.input.check_ready()

In [ ]:
exe = m.run(how='foreground')

In [ ]:
exe.status

In [ ]:
exe.output[0].plot()

In [ ]:
exe.output[0].equilibrium_volume

In [ ]:
exe.output[0].get_structure().get_volume()

In [ ]:
exe.output[0].get_structure()

## Again but execute children as background processes, but keep the task itself blocking

In [ ]:
m = MurnaghanTask()

In [ ]:
m.input.task = AseStaticTask()
m.input.task.input.calculator = MorsePotential()
m.input.structure = bulk("Fe", a=1.2)

In [ ]:
m.input.task.input

In [ ]:
m.input.set_strain_range(.6, 1000)

In [ ]:
m.input.child_executor = ProcessExecutor

In [ ]:
exe = m.run(how='foreground')

In [ ]:
exe._run_machine.state

In [ ]:
exe._run_time

In [ ]:
exe._collect_time

In [ ]:
exe.output[0].plot()

In [ ]:
exe.output[0].get_structure().get_volume()

In [ ]:
exe.output[0].equilibrium_volume

## Again but execute everything in the background.

In [ ]:
m = MurnaghanTask()

In [ ]:
m.input.task = AseStaticTask()
m.input.task.input.calculator = MorsePotential()
m.input.structure = bulk("Fe", a=1.2)

In [ ]:
m.input.task.input

In [ ]:
m.input.set_strain_range(.5, 100)

Use the threading backend just to show off.

In [ ]:
from pyiron_contrib.tinybase.executor import BackgroundExecutor

m.input.child_executor = BackgroundExecutor

In [ ]:
exe = m.run(how='background')

In [ ]:
exe.wait()

In [ ]:
exe._run_machine.state

In [ ]:
exe._run_time

In [ ]:
exe._collect_time

In [ ]:
exe.output[0].plot()

# Combine Minimize and Murnaghan

In [ ]:
m = MurnaghanTask()

In [ ]:
m.input.task = AseMinimizeTask()
m.input.task.input.calculator = MorsePotential()
m.input.task.input.max_steps = 100
m.input.task.input.output_steps = 10
m.input.task.input.ionic_force_tolerance = 1e-6
m.input.task.input.lbfgs()

m.input.structure = bulk("Fe", a=1.2)

In [ ]:
m.input.set_strain_range(.5, 500)

In [ ]:
m.input.child_executor = ProcessExecutor

In [ ]:
exe = m.run()

In [ ]:
exe._run_machine.state

In [ ]:
exe.status

In [ ]:
exe.output[0].energies[:10]

In [ ]:
exe.output[0].plot()